<a href="https://colab.research.google.com/github/vitaldb/respfailure/blob/main/respfailure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction Model for Postoperative Respiratory Failure

## Upgrade XGboost library

In [1]:
# latest version xgboost required
!pip uninstall xgboost -y
!pip install xgboost

Found existing installation: xgboost 0.90
Uninstalling xgboost-0.90:
  Successfully uninstalled xgboost-0.90
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 255.9 MB 34 kB/s 


## Download model and sample files

In [12]:
!wget https://github.com/vitaldb/respfailure/raw/main/model.json
!wget https://github.com/vitaldb/respfailure/raw/main/sample.csv

--2022-11-24 15:51:11--  https://github.com/vitaldb/respfailure/raw/main/model.json
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vitaldb/respfailure/main/model.json [following]
--2022-11-24 15:51:11--  https://raw.githubusercontent.com/vitaldb/respfailure/main/model.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190490 (186K) [text/plain]
Saving to: ‘model.json’

model.json          100%[===================>] 186.03K  --.-KB/s    in 0.03s   

2022-11-24 15:51:11 (6.66 MB/s) - ‘model.json’ saved [190490/190490]

--2022-11-24 15:51:11--  https://github.com/vitaldb/respfailure/raw/main/sam

## Sample code for single patient

In [13]:
import xgboost as xgb
import numpy as np
import sys
import pickle

# sample values
alb = 3.4
cr = 4.16
glu = 112
wbc = 6.59
bmi = 24
ptinr = 0.89
age = 78
andur = 105

model = xgb.Booster()
model.load_model('model.json')
x = np.array([alb, cr, glu, wbc, bmi, ptinr, age, andur]).astype(float)
x[x == 0] = np.nan  # use zero for missing value
x = x[None, ...]
y = model.predict(xgb.DMatrix(x))[0]
print(f'Risk={y * 100:.2f}%')

Risk=0.99%


## Sample code using CSV file for multiple patients

In [14]:
import xgboost as xgb
import numpy as np
import pandas as pd

model = xgb.Booster()
model.load_model('model.json')

df = pd.read_csv('sample.csv')
x = df[['alb', 'cr', 'glu', 'wbc', 'bmi', 'ptinr', 'age', 'andur']].astype(float).values
df['pred'] = model.predict(xgb.DMatrix(x))
df.to_csv('prediction.csv')
print(df)

    no  andur  age    wbc  ptinr  glu  alb   bmi    cr      pred
0    1    410   40   4.90   1.04   94  4.4  22.5  0.74  0.001270
1    2    100   30   4.20   1.01   96  4.6  19.1  0.66  0.000713
2    3    130   53   5.47   0.98   97  4.2  22.2  0.65  0.000731
3    4    295   59   5.67   1.07   99  4.6  22.4  0.70  0.001703
4    5    260   58   5.34   1.11  103  4.5  26.6  0.87  0.002252
5    6    510   69   4.55   0.96   84  4.0  23.6  0.73  0.008715
6    7    225   66   4.54   1.12  217  3.5  20.1  0.87  0.016042
7    8    355   25  11.10   1.09  135  4.7  24.8  0.93  0.004597
8    9    305   83   8.00   1.10   83  3.2  23.2  0.57  0.030371
9   10     65   56   4.47   0.92  111  4.4  21.2  0.89  0.000767
10  11    150   53  12.80   1.22  141  2.3  33.0  0.53  0.018343
11  12    250   63   4.59   0.97  252  4.5  24.6  0.85  0.002531
12  13    105   62   5.83   1.01  167  4.1  24.7  0.79  0.001140
13  14    115   56   5.58   1.01   83  3.8  21.8  1.05  0.001259
14  15    120   61   8.64